# Модуль В — Моделирование и прогнозирование

- Часть 1: Анализ моделей и важности признаков
- Часть 2: Прогнозирование динамики рисков (Prophet)
- Часть 3: Визуализация карт прогнозов

**Вход:** `labeled_dataset.csv` из Модуля Б

In [ ]:
# ===== Часть 1: Загрузка и анализ модели =====
import os
import pandas as pd
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, RocCurveDisplay
import numpy as np

# Настройки путей
MODEL_PATH = 'models/best_model.pkl'
DATA_PATH = 'labeled_dataset.csv'

# Проверка наличия файлов
if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError(f"Модель не найдена: {MODEL_PATH}")
if not os.path.exists(DATA_PATH):
    raise FileNotFoundError(f"Данные не найдены: {DATA_PATH}")

# Загрузка данных
df = pd.read_csv(DATA_PATH, encoding='utf-8')
model = joblib.load(MODEL_PATH)

# Формирование признаков
feature_cols = [c for c in df.columns if c not in ['risk_level', 'target', 'cluster_id', 'evacuation_difficulty']]
if not feature_cols:
    raise ValueError("Не найдено подходящих признаков для анализа")

X = df[feature_cols].copy()

# Кодируем категориальные признаки, если есть
for col in X.select_dtypes(include=['object']).columns:
    X[col] = X[col].astype('category').cat.codes

# Целевая переменная
if 'risk_level' in df.columns:
    y = (df['risk_level'] == 'High').astype(int)
elif 'target' in df.columns:
    y = df['target']
else:
    raise ValueError("Нет ни 'risk_level', ни 'target' в данных")

# Приведение типов
X = X.fillna(0)  # на случай пропусков

# Важность признаков (только для tree-based моделей)
if hasattr(model, 'feature_importances_'):
    fi = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    plt.figure(figsize=(8, 5))
    sns.barplot(x=fi.values, y=fi.index)
    plt.title('Важность признаков')
    plt.xlabel('Важность')
    plt.tight_layout()
    plt.savefig('models/feature_importance.png', dpi=150, bbox_inches='tight')
    plt.show()

# Матрица ошибок
y_pred = model.predict(X)
cm = confusion_matrix(y, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Low', 'High'], yticklabels=['Low', 'High'])
plt.title('Матрица ошибок')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.tight_layout()
plt.savefig('models/confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

# ROC-кривая
try:
    RocCurveDisplay.from_estimator(model, X, y)
    plt.title('ROC-кривая на всём датасете')
    plt.savefig('models/roc_curve.png', dpi=150, bbox_inches='tight')
    plt.show()
except Exception as e:
    print(f"⚠️ Не удалось построить ROC-кривую: {e}")

In [ ]:
# ===== Часть 2: Прогноз временных рядов (Prophet) =====
from prophet import Prophet
import matplotlib.pyplot as plt
import os

# Проверяем наличие нужных колонок
if 'risk_level' not in df.columns or 'date' not in df.columns:
    print("⚠️ Нет данных для прогноза: нужны 'risk_level' и 'date'")
else:
    # Фильтруем только высокий риск
    cluster_risk = df[df['risk_level'] == 'High'].copy()
    
    # Парсим дату
    cluster_risk['date'] = pd.to_datetime(cluster_risk['date'], errors='coerce')
    cluster_risk = cluster_risk.dropna(subset=['date'])
    cluster_risk['year'] = cluster_risk['date'].dt.year
    
    # Создаём папку для карт
    os.makedirs('maps', exist_ok=True)
    
    # Группируем по кластеру и году
    for cluster in cluster_risk['cluster_id'].dropna().unique():
        grp = cluster_risk[cluster_risk['cluster_id'] == cluster]
        yearly_counts = grp.groupby('year').size().reset_index(name='count')
        yearly_counts.columns = ['ds', 'y']
        
        # Требуется минимум 2 точки для обучения
        if len(yearly_counts) < 2:
            print(f"ℹ️ Недостаточно данных для прогноза кластера {int(cluster)}: {len(yearly_counts)} точек")
            continue
        
        # Обучение модели Prophet
        model = Prophet(
            yearly_seasonality=True,
            interval_width=0.95,
            changepoint_prior_scale=0.05
        )
        model.fit(yearly_counts)
        
        # Прогноз на 5 лет вперёд
        future = model.make_future_dataframe(periods=5, freq='Y')
        forecast = model.predict(future)
        
        # Визуализация
        fig = model.plot(forecast)
        plt.title(f'Прогноз опасных событий: кластер {int(cluster)}', fontsize=14)
        plt.xlabel('Год')
        plt.ylabel('Число инцидентов')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.savefig(f'maps/forecast_cluster_{int(cluster)}.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        # Сохраняем данные прогноза
        forecast.to_csv(f'maps/forecast_cluster_{int(cluster)}_data.csv', index=False, encoding='utf-8')
        
        print(f"✅ Прогноз для кластера {int(cluster)} сохранён: карта и CSV")

In [ ]:
# ===== Часть 3: Карта прогноза =====
import folium
import os
import numpy as np

# Палитра цветов для кластеров (расширена)
palette = {
    0: 'gray',      # Неизвестный / низкий риск
    1: 'red',       # Высокий риск
    2: 'orange',    # Средний риск
    3: 'green',     # Низкий риск
    4: 'blue',      # Дополнительный
    5: 'purple'     # Дополнительный
}

# Создаём папку, если её нет
os.makedirs('maps', exist_ok=True)

print("🗺️ Генерация интерактивных карт...")

for cluster in df['cluster_id'].dropna().unique():
    cluster_data = df[df['cluster_id'] == cluster][['lat', 'lon']].dropna()
    
    if cluster_data.empty:
        print(f"ℹ️ Нет координат для кластера {int(cluster)} — пропущено")
        continue
    
    # Центр карты — среднее по широте/долготе
    center_lat = cluster_data['lat'].mean()
    center_lon = cluster_data['lon'].mean()
    coords = cluster_data.values

    # Создаём карту
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=10,
        tiles='OpenStreetMap',
        control_scale=True
    )

    # Добавляем точки
    for lat, lon in coords:
        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color=palette.get(int(cluster) % len(palette), 'gray'),
            fill=True,
            fill_color=palette.get(int(cluster) % len(palette), 'gray'),
            fill_opacity=0.7,
            weight=2,
            popup=f"📍 Кластер: {int(cluster)}<br>Широта: {lat:.5f}<br>Долгота: {lon:.5f}"
        ).add_to(m)

    # Сохраняем HTML-карту
    map_path = f'maps/forecast_cluster_{int(cluster)}.html'
    m.save(map_path)
    print(f"✅ Карта сохранена: {map_path}")

print("🎉 Все карты успешно сгенерированы!")